<a href="https://colab.research.google.com/github/Antonfsp/audio_classification_project/blob/master/script_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython==5.5

In [ ]:
!pip install fastaudio
!pip install fastai==2.3.1

In [ ]:
!pip install torchaudio==0.8.1

In [ ]:
!tar -xzvf '/content/drive/MyDrive/Medley-solos-DB.tar.gz' -C '/content'

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

import shutil,os

In [ ]:
path = Path()

wavs = path.ls()
wavs

In [ ]:
wavs.remove(Path('.config'))
wavs.remove(Path('drive'))
wavs.remove(Path('sample_data'))

In [ ]:
# creating the directory "data" and its subdirectories
os.mkdir("data")
for subset in ["train","validation","test"]:
    os.makedirs("data/%s" % subset)

# Moving the files to the corresponding directories
for file in wavs:
    if "test" in str(file.name):
        shutil.move(str(file),"data/test")
    elif "validation" in str(file.name):
        shutil.move(str(file),"data/validation")
    else:
        shutil.move(str(file),"data/train") 

In [ ]:
wavs = []
for sub in Path("data").ls():
        wavs = wavs + Path(str(sub)).ls()
wavs 

In [ ]:
for file in wavs:
    if "._" in file.name:
        file.unlink()

In [ ]:
wavs = []
for sub in Path("data").ls():
        wavs = wavs + Path(str(sub)).ls()
wavs 

In [ ]:
def get_instrument_tag(file):
    instruments_ids = {0:'clarinet', 1:'distorted electric guitar', 2:'female singer', 3:'flute', 4:'piano',
5:'tenor saxophone', 6:'trumpet', 7:'violin'}
    return instruments_ids[int(file.name[-42])]

In [ ]:
for subset in ["train","validation","test"]:
    instruments_ids = {0:'clarinet', 1:'distorted electric guitar', 2:'female singer', 3:'flute', 4:'piano',
5:'tenor saxophone', 6:'trumpet', 7:'violin'}
    for instrument in instruments_ids.values():
        os.makedirs("data/%s/%s" % (subset,instrument)) 

# Moving the files to the corresponding directories
for subset in ["train","validation","test"]:
    [shutil.move(str(file),"data/%s/%s" %(subset,get_instrument_tag(file))) for file in wavs if subset in str(file.name)]

In [ ]:
wavs = []
for sub in Path("data").ls():
    for instrument in Path(str(sub)).ls():
        wavs = wavs + Path(str(instrument)).ls()
wavs

In [ ]:
cfg = AudioConfig.BasicMelSpectrogram() # with default torchaudio parameters
audio2melspec = AudioToSpec.from_cfg(cfg)

In [ ]:
aud_digit = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_items=get_audio_files, 
                 splitter=GrandparentSplitter(train_name='train', valid_name='validation'),
                 item_tfms = audio2melspec,
                 get_y=parent_label)

In [ ]:
path = Path("data")
dls = aud_digit.dataloaders(path, bs=8)

In [ ]:
def alter_learner(learn, n_channels=1):
  "Adjust a `Learner`'s model to accept `1` channel"
  layer = learn.model[0][0]
  layer.in_channels=n_channels
  layer.weight = nn.Parameter(layer.weight[:,1,:,:].unsqueeze(1))
  learn.model[0][0] = layer

In [ ]:
learn = Learner(dls, xresnet18(), CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
n_c = dls.one_batch()[0].shape[1]; n_c

In [ ]:
alter_learner(learn, n_c)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, 1e-3)

In [ ]:
learn.unfreeze()
learn.save

In [ ]:
learn.export("models/3_epochs.pkl")

In [ ]:
learn.lr_find()

In [28]:
path = Path()
learn = load_learner(path/'models/3_epochs.pkl')

In [30]:
learn.fit_one_cycle(2, slice(1e-9,1e-8))

epoch,train_loss,valid_loss,accuracy,time
0,nan,None,None,00:00
1,nan,None,None,00:00


/usr/local/lib/python3.7/dist-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [29]:
learn.predict(Path("data/test/clarinet/Medley-solos-DB_test-0_003d41a8-afad-501f-fbc3-d406dc694ceb.wav"))

('female singer',
 tensor(2),
 tensor([1.5588e-02, 8.5249e-04, 9.8091e-01, 2.4396e-06, 1.7830e-05, 6.2872e-06,
         7.9937e-07, 2.6134e-03, 5.0985e-09, 6.2148e-09, 3.7546e-09, 1.4680e-09,
         4.3123e-09, 5.7314e-09, 2.1689e-09, 6.6095e-09, 7.0677e-09, 8.3343e-09,
         5.1566e-09, 1.1667e-08, 5.5544e-09, 1.2441e-08, 1.5108e-08, 5.6831e-09,
         1.4059e-08, 1.4616e-08, 2.3192e-09, 8.2835e-09, 3.6407e-09, 1.8976e-09,
         1.6681e-09, 4.9342e-09, 7.8736e-09, 2.5538e-09, 6.3085e-09, 7.0833e-09,
         7.7605e-09, 1.5003e-08, 1.6866e-08, 2.4247e-09, 5.2133e-09, 4.1782e-09,
         1.4305e-08, 3.7423e-09, 6.7318e-09, 3.2345e-09, 8.3790e-09, 1.0000e-08,
         1.2932e-08, 5.3830e-09, 3.5004e-09, 2.3114e-09, 2.6001e-08, 3.3327e-09,
         5.8662e-09, 5.5533e-09, 8.5497e-09, 4.5279e-09, 3.2377e-09, 9.2508e-09,
         5.1873e-09, 7.3348e-09, 2.8271e-09, 5.7857e-09, 2.5167e-09, 1.7998e-08,
         3.0075e-09, 3.5864e-09, 1.9115e-09, 3.2905e-09, 9.0254e-09, 8.5082e-09